# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np

TensorFlow 2.x selected.


In [2]:
tf.__version__

'2.1.0'

In [0]:
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [4]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [0]:
# Test data
trainX1 = x_train[np.extract((y_train<5), x_train)]
trainY1 = y_train[y_train<5]

trainX2 = x_train[np.extract((y_train>=5), x_train)]
trainY2 = y_train[y_train>=5]

# Test data
testX1 = x_test[np.extract((y_test<5), x_test)]
testY1 = y_test[y_test<5]

testX2 = x_test[np.extract((y_test>=5), x_test)]
testY2 = y_test[y_test>=5]

In [6]:
print(trainX1.shape)
print(trainY1.shape)
print(trainX2.shape)
print(trainY2.shape)

(25000, 32, 32, 3)
(25000,)
(25000, 32, 32, 3)
(25000,)


In [7]:
print(testX1.shape)
print(testY1.shape)
print(testX2.shape)
print(testY2.shape)

(5000, 32, 32, 3)
(5000,)
(5000, 32, 32, 3)
(5000,)


In [0]:
trainX1 = np.asarray(trainX1).reshape((-1, 32, 32, 3))
testX1 = np.asarray(testX1).reshape((-1, 32, 32, 3))
trainX2 = np.asarray(trainX2).reshape((-1, 32, 32, 3))
testX2 = np.asarray(testX2).reshape((-1, 32, 32, 3))

In [0]:
trainX1 = np.array(trainX1).astype('float32')
testX1 = np.array(testX1).astype('float32')

trainX1 /= 255
testX1 /= 255

In [0]:
trainX2 = np.array(trainX2).astype('float32')
testX2 = np.array(testX2).astype('float32')

trainX2 /= 255
testX2/= 255

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
trainY1 = tf.keras.utils.to_categorical(trainY1, num_classes=5)
testY1 = tf.keras.utils.to_categorical(testY1, num_classes=5)

In [0]:
trainY2 = tf.keras.utils.to_categorical(trainY1, num_classes=5)
testY2 = tf.keras.utils.to_categorical(testY1, num_classes=5)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,vertical_flip=True,zoom_range=0.3)

In [0]:
#Clear any existing model in memory
tf.keras.backend.clear_session()

#Initialize model, reshape & normalize data
model = tf.keras.models.Sequential()

#Add first convolutional layer    
model.add(tf.keras.layers.Conv2D(128,kernel_size= (3, 3), activation = 'relu',input_shape=trainX1.shape[1:]))

#normalize data
model.add(tf.keras.layers.BatchNormalization())

#Add Second convolutional layer    
model.add(tf.keras.layers.Conv2D(128,kernel_size=(3, 3), activation = 'relu'))

#Add maxpool layer
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

#Add a dropout layer
model.add(tf.keras.layers.Dropout(0.2))

In [0]:
#Add third convolutional layer
model.add(tf.keras.layers.Conv2D(256,kernel_size= (3, 3),activation='relu'))

#Add maxpool layer
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

#normalize data
model.add(tf.keras.layers.BatchNormalization())

#Add fourth convolutional layer
model.add(tf.keras.layers.Conv2D(256,kernel_size= (3,3),activation='relu'))

#Add max pool layer
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

In [0]:
#Flatten out the data
model.add(tf.keras.layers.Flatten())

# Adding a dense layer
model.add(tf.keras.layers.Dense(128,activation = 'relu'))

#Adding a drouput layer
model.add(tf.keras.layers.Dropout(0.2))

#Adding output layer
model.add(tf.keras.layers.Dense(5,activation='softmax'))

In [0]:
sgd = tf.keras.optimizers.SGD(lr = 0.1, decay=1e-6, momentum=0.7,nesterov=True)

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 30, 128)       512       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 128)       147584    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 256)       295168    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 256)         0

In [0]:
#Build training generator. 
train_generator = datagen.fit(trainX1)

#Build test generator
test_generator = datagen.fit(testX1)

In [68]:
model.fit_generator(datagen.flow(trainX1, trainY1, batch_size=32),
                    epochs=10,
                        validation_data=(testX1, testY1))

  ...
    to  
  ['...']
Train for 782 steps, validate on 5000 samples
Epoch 1/10
782/782 [==============================] - 18s 23ms/step - loss: 1.6348 - accuracy: 0.1927 - val_loss: 1.6094 - val_accuracy: 0.2000
Epoch 2/10
782/782 [==============================] - 17s 22ms/step - loss: 1.6096 - accuracy: 0.1964 - val_loss: 1.6094 - val_accuracy: 0.2000
Epoch 3/10
782/782 [==============================] - 18s 23ms/step - loss: 1.6121 - accuracy: 0.1954 - val_loss: 1.6094 - val_accuracy: 0.2000
Epoch 4/10
782/782 [==============================] - 18s 23ms/step - loss: 1.6102 - accuracy: 0.1987 - val_loss: 1.6095 - val_accuracy: 0.2000
Epoch 5/10
782/782 [==============================] - 18s 23ms/step - loss: 1.6096 - accuracy: 0.1986 - val_loss: 1.6095 - val_accuracy: 0.2000
Epoch 6/10
782/782 [==============================] - 18s 22ms/step - loss: 1.6095 - accuracy: 0.1993 - val_loss: 1.6095 - val_accuracy: 0.2000
Epoch 7/10
782/782 [==============================] - 18s 23ms/st

In [20]:
model.output

<tf.Tensor 'dense_1/Identity:0' shape=(None, 5) dtype=float32>

In [0]:
new_model=model

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [22]:
   
import pandas as pd
pd.set_option('max_colwidth', -1)
layers = [(layer, layer.name, layer.trainable) for layer in new_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable']) 

,Layer Type,Layer Name,Layer Trainable
0,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f73c7d98f60>,conv2d,True
1,<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f744c48eba8>,batch_normalization,True
2,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f744c4ae828>,conv2d_1,True
3,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f744c4b2e48>,max_pooling2d,True
4,<tensorflow.python.keras.layers.core.Dropout object at 0x7f73c01157f0>,dropout,True
5,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f73c7d98780>,conv2d_2,True
6,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f73c7d987b8>,max_pooling2d_1,True
7,<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f73c0115ba8>,batch_normalization_1,True
8,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f73c01307b8>,conv2d_3,True
9,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f73c0136710>,max_pooling2d_2,True


In [23]:
set_trainable = False
for layer in model.layers[:11]:
    layer.trainable = False

layers = [(layer, layer.name, layer.trainable) for layer in model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable']) 

,Layer Type,Layer Name,Layer Trainable
0,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f73c7d98f60>,conv2d,False
1,<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f744c48eba8>,batch_normalization,False
2,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f744c4ae828>,conv2d_1,False
3,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f744c4b2e48>,max_pooling2d,False
4,<tensorflow.python.keras.layers.core.Dropout object at 0x7f73c01157f0>,dropout,False
5,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f73c7d98780>,conv2d_2,False
6,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f73c7d987b8>,max_pooling2d_1,False
7,<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f73c0115ba8>,batch_normalization_1,False
8,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f73c01307b8>,conv2d_3,False
9,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f73c0136710>,max_pooling2d_2,False


### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [0]:
#new_model=model
new_model.pop()

In [25]:
#Check if layers frozen
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 30, 128)       512       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 128)       147584    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 256)       295168    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 256)         0

In [0]:
new_model.add(tf.keras.layers.Dense(128,activation='relu'))
#Add Dropout
new_model.add(tf.keras.layers.Dropout(0.25))

#Add Output Layer
new_model.add(tf.keras.layers.Dense(5,activation='softmax'))


In [0]:
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 30, 128)       512       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 128)       147584    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 256)       295168    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 256)         0

In [29]:
new_model.fit(trainX2, trainY2, epochs=10, shuffle=True, batch_size=64, validation_data=(testX2, testY2), verbose=2)

Train on 25000 samples, validate on 5000 samples
Epoch 1/10


InvalidArgumentError: ignored

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [0]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [0]:
twenty_train.target

In [0]:
twenty_train.target_names

In [0]:
twenty_train.data[0:5]

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [0]:
vectorizer = TfidfVectorizer(stop_words='english', min_df=0.001, max_df=0.20)
vectors = vectorizer.fit_transform(twenty_train.data)
vectors_test = vectorizer.transform(twenty_test.data)
x1 = vectors
y1 = twenty_train.target
x2 = vectors_test
y2 = twenty_test.target
x = np.array(np.r_[x1.todense(), x2.todense()])
y = np.r_[y1, y2]

In [0]:
pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [0]:
# use logistic regression with text column only
logreg = LogisticRegression(C=1e9)
logreg.fit(vectors, y1)
y_pred_class = logreg.predict(vectors_test)
print(metrics.accuracy_score(y2, y_pred_class))